# Importação dos módulos

In [322]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
nlp = spacy.load('pt_core_news_md')
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TRT05\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Importação dos dados

In [323]:
df = pd.read_csv('leis.csv')
df.head()

,lei,ementa
0,"Lei nº 14.464, de 1º.11.2022Publicada no DOU d...",Autoriza o Poder Executivo federal a doar 20 (...
1,"Lei nº 14.463, de 26.10.2022Publicada no DOU d...",Dispõe sobre a reabertura do prazo para opção ...
2,"Lei nº 14.462, de 26.10.2022Publicada no DOU d...","Altera as Leis nºs 11.977, de 7 de julho de 20..."
3,"Lei nº 14.461, de 25.10.2022Publicada no DOU d...",Autoriza a prorrogação de contratos por tempo ...
4,"Lei nº 14.460, de 25.10.2022Publicada no DOU d...",Transforma a Autoridade Nacional de Proteção d...


# Tratamento dos dados

## Separando identificador da lei

In [324]:
df['lei'] = df['lei'].str.split(',').str[0]
df.head()

,lei,ementa
0,Lei nº 14.464,Autoriza o Poder Executivo federal a doar 20 (...
1,Lei nº 14.463,Dispõe sobre a reabertura do prazo para opção ...
2,Lei nº 14.462,"Altera as Leis nºs 11.977, de 7 de julho de 20..."
3,Lei nº 14.461,Autoriza a prorrogação de contratos por tempo ...
4,Lei nº 14.460,Transforma a Autoridade Nacional de Proteção d...


## Tratando a coluna de ementa

### Removendo números

In [325]:
df['ementa_tratada'] = df['ementa'].str.replace('\d+', '')
df.head()

C:\Users\TRT05\AppData\Local\Temp\ipykernel_12536\898448057.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ementa_tratada'] = df['ementa'].str.replace('\d+', '')


,lei,ementa,ementa_tratada
0,Lei nº 14.464,Autoriza o Poder Executivo federal a doar 20 (...,Autoriza o Poder Executivo federal a doar (vi...
1,Lei nº 14.463,Dispõe sobre a reabertura do prazo para opção ...,Dispõe sobre a reabertura do prazo para opção ...
2,Lei nº 14.462,"Altera as Leis nºs 11.977, de 7 de julho de 20...","Altera as Leis nºs ., de de julho de , ., de ..."
3,Lei nº 14.461,Autoriza a prorrogação de contratos por tempo ...,Autoriza a prorrogação de contratos por tempo ...
4,Lei nº 14.460,Transforma a Autoridade Nacional de Proteção d...,Transforma a Autoridade Nacional de Proteção d...


### Removendo capitalização

In [326]:
df['ementa_tratada'] = df['ementa_tratada'].str.lower()
df.head()

,lei,ementa,ementa_tratada
0,Lei nº 14.464,Autoriza o Poder Executivo federal a doar 20 (...,autoriza o poder executivo federal a doar (vi...
1,Lei nº 14.463,Dispõe sobre a reabertura do prazo para opção ...,dispõe sobre a reabertura do prazo para opção ...
2,Lei nº 14.462,"Altera as Leis nºs 11.977, de 7 de julho de 20...","altera as leis nºs ., de de julho de , ., de ..."
3,Lei nº 14.461,Autoriza a prorrogação de contratos por tempo ...,autoriza a prorrogação de contratos por tempo ...
4,Lei nº 14.460,Transforma a Autoridade Nacional de Proteção d...,transforma a autoridade nacional de proteção d...


### Removendo pontuação

In [327]:
df['ementa_tratada'] = df['ementa_tratada'].str.replace('[^\w\s]', '')
df.head()

C:\Users\TRT05\AppData\Local\Temp\ipykernel_12536\1053391803.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ementa_tratada'] = df['ementa_tratada'].str.replace('[^\w\s]', '')


,lei,ementa,ementa_tratada
0,Lei nº 14.464,Autoriza o Poder Executivo federal a doar 20 (...,autoriza o poder executivo federal a doar vin...
1,Lei nº 14.463,Dispõe sobre a reabertura do prazo para opção ...,dispõe sobre a reabertura do prazo para opção ...
2,Lei nº 14.462,"Altera as Leis nºs 11.977, de 7 de julho de 20...",altera as leis nºs de de julho de de de j...
3,Lei nº 14.461,Autoriza a prorrogação de contratos por tempo ...,autoriza a prorrogação de contratos por tempo ...
4,Lei nº 14.460,Transforma a Autoridade Nacional de Proteção d...,transforma a autoridade nacional de proteção d...


### Removendo stopwords

In [328]:
stop = stopwords.words('portuguese')
stop_custom = ['autoriza', 'dispõe', 'altera', 'institui', 'estabelece', 'define', 'cria', 'institui', 'declara', 'denomina', 'reconhece', 'abre', 'cria', 'nºs']
stop.extend(stop_custom)

df['ementa_tratada'] = df['ementa_tratada'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

df.head()

,lei,ementa,ementa_tratada
0,Lei nº 14.464,Autoriza o Poder Executivo federal a doar 20 (...,poder executivo federal doar vinte viaturas op...
1,Lei nº 14.463,Dispõe sobre a reabertura do prazo para opção ...,sobre reabertura prazo opção regime previdênci...
2,Lei nº 14.462,"Altera as Leis nºs 11.977, de 7 de julho de 20...",leis julho janeiro novembro agosto dispõem sob...
3,Lei nº 14.461,Autoriza a prorrogação de contratos por tempo ...,prorrogação contratos tempo determinado âmbito...
4,Lei nº 14.460,Transforma a Autoridade Nacional de Proteção d...,transforma autoridade nacional proteção dados ...


### Lematização

In [329]:
df['ementa_tratada'] = df['ementa_tratada'].apply(lambda x: " ".join([y.lemma_ for y in nlp(x)]))
df.head()

,lei,ementa,ementa_tratada
0,Lei nº 14.464,Autoriza o Poder Executivo federal a doar 20 (...,poder executivo federal doar vinte viatura ope...
1,Lei nº 14.463,Dispõe sobre a reabertura do prazo para opção ...,sobre reabertura prazo opção regime previdênci...
2,Lei nº 14.462,"Altera as Leis nºs 11.977, de 7 de julho de 20...",lei julho janeiro novembro agosto dispor sobre...
3,Lei nº 14.461,Autoriza a prorrogação de contratos por tempo ...,prorrogação contrato tempo determinar âmbito f...
4,Lei nº 14.460,Transforma a Autoridade Nacional de Proteção d...,transformar autoridade nacional proteção dar a...


# Sistema de recomendação

## doc2vec

In [330]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['ementa_tratada'].str.split(' '))]

model = Doc2Vec(documents, vector_size = 110, window = 2, min_count = 1)

In [331]:
def recomenda_leis(index):
  documentos = []
  for i in model.docvecs.most_similar(index):
    documentos.append({ 'lei': df.iloc[i[0]].lei, 'ementa': df.iloc[i[0]].ementa, 'similaridade': i[1] })
  df_leis_rec = pd.DataFrame(documentos)
  return df_leis_rec

In [332]:
print(df.iloc[6395].ementa)
recomenda_leis(6395)

Dispõe sobre o Estatuto da Criança e do Adolescente e dá outras providências.


C:\Users\TRT05\AppData\Local\Temp\ipykernel_12536\753154412.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  for i in model.docvecs.most_similar(index):


,lei,ementa,similaridade
0,10.810,Abre aos Orçamentos Fiscal e da Seguridade Soc...,0.868038
1,9.140,Reconhece como mortas pessoas desaparecidas em...,0.855977
2,13.321,Altera o soldo e o escalonamento vertical dos ...,0.851969
3,12.258,"Altera o Decreto-Lei n o 2.848, de 7 de dezemb...",0.850077
4,12.959,"Altera a Lei no 7.678, de 8 de novembro de 198...",0.846861
5,12.587,Institui as diretrizes da Política Nacional de...,0.843610
6,Lei nº 13.812,Institui a Política Nacional de Busca de Pesso...,0.841634
7,12.299,Dispõe sobre medidas de prevenção e repressão ...,0.839748
8,13.092,Dispõe sobre o subsídio do Procurador-Geral da...,0.838157
9,13.322,"Altera as Leis n os 9.615, de 24 de março de 1...",0.837175


In [333]:
print(df.iloc[5798].ementa)
recomenda_leis(5798)

Regulamenta o art. 37, inciso XXI, da Constituição Federal, institui normas para licitações e contratos da Administração Pública e dá outras providências.
Vide texto compilado
Mensagem de veto


C:\Users\TRT05\AppData\Local\Temp\ipykernel_12536\753154412.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  for i in model.docvecs.most_similar(index):


,lei,ementa,similaridade
0,12.858,Dispõe sobre a destinação para as áreas de edu...,0.972731
1,9.882,Dispõe sobre o processo e julgamento da argüiç...,0.970501
2,10.220,Institui normas gerais relativas à atividade d...,0.969917
3,Lei nº 14.024,"Altera a Lei nº 10.260, de 12 de julho de 2001...",0.967000
4,10.164,Prorroga o prazo para as ratificações de conce...,0.966965
5,Lei nº 13.894,"Altera a Lei nº 11.340, de 7 de agosto de 2006...",0.966238
6,11.577,Torna obrigatória a divulgação pelos meios que...,0.966145
7,11.101,"Regula a recuperação judicial, a extrajudicial...",0.965988
8,12.232,Dispõe sobre as normas gerais para licitação e...,0.965986
9,11.900,"Altera dispositivos do Decreto-Lei n o 3.689, ...",0.963818


In [334]:
print(df.iloc[3944].ementa)
recomenda_leis(3944)

Institui, no âmbito da União, Estados, Distrito Federal e Municípios, nos termos do art. 37, inciso XXI, da Constituição Federal, modalidade de licitação denominada pregão, para aquisição de bens e serviços comuns, e dá outras providências. Mensagem de veto


C:\Users\TRT05\AppData\Local\Temp\ipykernel_12536\753154412.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  for i in model.docvecs.most_similar(index):


,lei,ementa,similaridade
0,13.123,Regulamenta o inciso II do § 1 o e o § 4 o do ...,0.983526
1,Lei nº 14.451,"Altera a Lei nº 10.406, de 10 de janeiro de 20...",0.980610
2,11.466,"Altera a Lei n o 7.210, de 11 de julho de 1984...",0.980254
3,Lei nº 14.369,"Altera a Lei Complementar nº 89, de 18 de feve...",0.979577
4,10.461,Acrescenta alínea ao inciso I do art. 23 da Le...,0.979233
5,Lei nº 14.309,"Altera a Lei nº 10.406, de 10 de janeiro de 20...",0.979061
6,11.705,"Altera a Lei n o 9.503, de 23 de setembro de 1...",0.978674
7,9.282,Reajusta a pensão especial concedida pela Lei ...,0.977793
8,Lei nº 13.912,"Altera a Lei nº 10.671, de 15 de maio de 2003 ...",0.977673
9,10.763,Acrescenta artigo ao Código Penal e modifica a...,0.977635


In [335]:
print(df.iloc[331].ementa)
recomenda_leis(331)

Lei de Licitações e Contratos Administrativos.  Mensagem de veto


C:\Users\TRT05\AppData\Local\Temp\ipykernel_12536\753154412.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  for i in model.docvecs.most_similar(index):


,lei,ementa,similaridade
0,11.577,Torna obrigatória a divulgação pelos meios que...,0.960021
1,10.220,Institui normas gerais relativas à atividade d...,0.955429
2,11.101,"Regula a recuperação judicial, a extrajudicial...",0.951411
3,11.804,Disciplina o direito a alimentos gravídicos e ...,0.945632
4,8.723,Dispõe sobre a redução de emissão de poluentes...,0.944147
5,10.814,Estabelece normas para o plantio e comercializ...,0.944070
6,Lei nº 13.869,Dispõe sobre os crimes de abuso de autoridade;...,0.942222
7,13.052,"Altera o art. 25 da Lei no 9.605, de 12 de fev...",0.940792
8,11.131,Autoriza a União a prestar auxílio financeiro ...,0.940684
9,12.744,Altera o art. 4 o e acrescenta art. 54-A à Lei...,0.940179


## TFIDF

In [336]:
vectorizer = TfidfVectorizer(min_df = 2, max_df = 0.7)
vectorized_data = vectorizer.fit_transform(df['ementa_tratada'])
print(vectorizer.get_feature_names())

['abastecimento', 'abc', 'aberto', 'abertura', 'aberturar', 'abgf', 'abin', 'abolição', 'abono', 'abrangir', 'abrangência', 'abril', 'abrir', 'absorção', 'abuso', 'academia', 'acelerar', 'aceleração', 'acerca', 'acessibilidade', 'acesso', 'acessível', 'acidente', 'acionário', 'acolhimento', 'acometir', 'acompanhamento', 'acompanhante', 'acompanhar', 'acondicionamento', 'acordo', 'acre', 'acrescentar', 'acrescer', 'acréscimo', 'acusar', 'acusação', 'adaptar', 'adaptação', 'adene', 'adensamento', 'adequado', 'adequar', 'adequação', 'adequála', 'adequálo', 'adesão', 'adicional', 'aditivo', 'adição', 'adjunto', 'administrador', 'administrativo', 'administração', 'admitir', 'adolescente', 'adotante', 'adotar', 'adoção', 'adquirente', 'adquirir', 'aduaneiro', 'adulto', 'advertêncio', 'advocacer', 'advocacia', 'advocaciageral', 'advogado', 'advogados', 'advogar', 'aeronave', 'aeronáutica', 'aeronáuticar', 'aeronáutico', 'aeroporto', 'aeroportuário', 'aeródromos', 'afastamento', 'afetação', 'a

c:\Users\TRT05\Desktop\recomenda-leis\.venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [337]:
tfidf_df = pd.DataFrame(vectorized_data.toarray(), columns = vectorizer.get_feature_names(), index = df.lei)
tfidf_df.head()

,abastecimento,abc,aberto,abertura,aberturar,abgf,abin,abolição,abono,abrangir,...,âmbito,égide,índice,índio,íntimo,óbito,óleo,órgão,único,útil
lei,,,,,,,,,,,,,,,,,,,,,
Lei nº 14.464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lei nº 14.463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lei nº 14.462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lei nº 14.461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.232558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lei nº 14.460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [340]:
similarity_array = linear_kernel(tfidf_df)
similarity_df = pd.DataFrame(similarity_array, index = tfidf_df.index, columns = tfidf_df.index)
similarity_df.head()


lei,Lei nº 14.464,Lei nº 14.463,Lei nº 14.462,Lei nº 14.461,Lei nº 14.460,Lei nº 14.459,Lei nº 14.458,Lei nº 14.457,Lei nº 14.456,Lei nº 14.455,...,7.656,7.655,7.654,7.653,7.652,7.651,7.650,7.649,7.648,7.647
lei,,,,,,,,,,,,,,,,,,,,,
Lei nº 14.464,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.019168,0.032818,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.036608,0.000000,0.000000,0.031804,0.000000
Lei nº 14.463,0.0,1.000000,0.067865,0.0,0.140613,0.081559,0.010760,0.087049,0.038707,0.073442,...,0.01730,0.024901,0.018763,0.102971,0.015867,0.095753,0.000000,0.000000,0.000000,0.174676
Lei nº 14.462,0.0,0.067865,1.000000,0.0,0.101759,0.018400,0.008573,0.058310,0.008759,0.011022,...,0.00000,0.000000,0.000000,0.060109,0.012642,0.066947,0.000000,0.000000,0.000000,0.162113
Lei nº 14.461,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.053481,0.000000,0.000000,0.117530,0.000000,0.000000,0.000000,0.000000
Lei nº 14.460,0.0,0.140613,0.101759,0.0,1.000000,0.051650,0.000000,0.070025,0.111218,0.034624,...,0.01736,0.052872,0.018828,0.250298,0.031903,0.038622,0.051091,0.014109,0.026791,0.120816


In [342]:
similarity_series_1 = similarity_df.loc['8.069']

ordered_similarities_1 = similarity_series_1.sort_values(ascending = False)

print(ordered_similarities_1[:10])

lei
8.069     1.000000
10.764    0.905539
13.440    0.796464
11.185    0.783934
13.046    0.710496
9.975     0.688904
11.259    0.650520
8.242     0.641091
13.010    0.637856
13.431    0.631484
Name: 8.069, dtype: float64


In [343]:
similarity_series_2 = similarity_df.loc['8.666']

ordered_similarities_2 = similarity_series_2.sort_values(ascending = False)

print(ordered_similarities_2[:10])

lei
8.666            1.000000
11.763           0.730420
8.883            0.704204
9.854            0.697700
11.783           0.663442
10.753           0.466182
9.718            0.457521
Lei nº 13.934    0.412002
8.448            0.403120
11.250           0.393647
Name: 8.666, dtype: float64


In [344]:
similarity_series_3 = similarity_df.loc['10.520']

ordered_similarities_3 = similarity_series_3.sort_values(ascending = False)

print(ordered_similarities_3[:10])

lei
10.520           1.000000
8.883            0.476660
9.854            0.404038
11.763           0.395220
8.666            0.389980
11.783           0.367009
12.153           0.316609
Lei nº 14.337    0.289524
12.052           0.289524
12.948           0.289524
Name: 10.520, dtype: float64


In [346]:
similarity_series_4 = similarity_df.loc['Lei nº 14.133']

ordered_similarities_4 = similarity_series_4.sort_values(ascending = False)

print(ordered_similarities_4[:10])

lei
Lei nº 14.133    1.000000
11.763           0.411671
13.445           0.397965
8.666            0.391097
9.854            0.379522
10.891           0.359474
9.601            0.359354
8.883            0.346584
Lei nº 14.065    0.335233
11.079           0.316834
Name: Lei nº 14.133, dtype: float64
